<h1>NIN Model</h1>
<p>The file is the NIN model we use. The training dataset is 'train.csv', without 'extra.csv'. In this case, we only run the epoch =  10, 
but we could make the epoch larger to get a higher accuracy. </p>
<p>In the submisstion in Kaggle, the epoch is 25.</p>

In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from utils import *
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

Using TensorFlow backend.


In [3]:
def dense_to_one_hot(labels_dense, num_classes=10):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    for i in range(labels_dense.shape[0]):
        labels_one_hot[i,labels_dense[i]]=1;
    return labels_one_hot

In [4]:
print("Loading data")
mat_train = loadmat("data/train_32x32.mat")
mat_train['X'] = mat_train['X'].transpose(3, 0, 1, 2)

x_train = mat_train['X']
y_train = mat_train['y'].reshape(-1, )
y_train[y_train == 10] = 0

del mat_train

mat_test = loadmat('data/test_32x32.mat')
mat_test['X'] = mat_test['X'].transpose(3, 0, 1, 2)
x_test = mat_test['X']
del mat_test

x_train_final=x_train[:,2:30,2:30,:]
x_test_final=x_test[:,2:30,2:30,:]

x_train, x_val, y_train, y_val = train_test_split(x_train_final, y_train, test_size=0.2, random_state=1)

print("Data loaded!")

Loading data
Data loaded!


In [8]:
batch_size = 256
num_classes = 10
epochs = 10

model = Sequential()

model.add(Conv2D(32, (5, 5), padding='VALID',
                 input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (1, 1)))
model.add(Conv2D(32, (1, 1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='VALID'))
model.add(Conv2D(64, (1, 1)))
model.add(Conv2D(64, (1, 1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (2, 2), padding='VALID'))
model.add(Conv2D(128, (1, 1)))
model.add(Conv2D(128, (1, 1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [6]:
y_train_final = dense_to_one_hot(y_train, num_classes)
y_val_final = dense_to_one_hot(y_val, num_classes)

In [9]:
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


print('Not using data augmentation.')
history=model.fit(x_train, y_train_final,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val_final),
          shuffle=True)



Not using data augmentation.
Train on 58605 samples, validate on 14652 samples
Epoch 1/10
58605/58605 [==============================] - 546s - loss: 0.9441 - acc: 0.6962 - val_loss: 0.5483 - val_acc: 0.8335
Epoch 2/10
58605/58605 [==============================] - 517s - loss: 0.5067 - acc: 0.8436 - val_loss: 0.4676 - val_acc: 0.8625
Epoch 3/10
58605/58605 [==============================] - 502s - loss: 0.4396 - acc: 0.8646 - val_loss: 0.4482 - val_acc: 0.8655
Epoch 4/10
58605/58605 [==============================] - 506s - loss: 0.4002 - acc: 0.8780 - val_loss: 0.3438 - val_acc: 0.9011
Epoch 5/10
58605/58605 [==============================] - 495s - loss: 0.3745 - acc: 0.8858 - val_loss: 0.3550 - val_acc: 0.8979
Epoch 6/10
58605/58605 [==============================] - 499s - loss: 0.3496 - acc: 0.8945 - val_loss: 0.3358 - val_acc: 0.9059
Epoch 7/10
58605/58605 [==============================] - 501s - loss: 0.3365 - acc: 0.8979 - val_loss: 0.3160 - val_acc: 0.9113
Epoch 8/10
58605/5

In [10]:
y_test_final=np.argmax(model.predict(x_test_final),axis=1)

In [11]:
plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['test', 'train'], loc='upper left')
plt.show()

NameError: name 'ModelName' is not defined

In [ ]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['test', 'train'], loc='upper left')
plt.show()

In [14]:
y_test=np.where(y_test_final>0,y_test_final,10)

In [15]:
import csv
with open("output_nin.csv","w",newline="") as datacsv:
    csvwriter = csv.writer(datacsv,dialect = ("excel"))
    csvwriter.writerow(['ImageId', 'label'])
    for i in range(len(y_test)):
        csvwriter.writerow((i, y_test[i]))